In [1]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy

In [3]:
paths_common = ['DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/OBJET/OBJET_COMPLET',
                'DONNEES/DESCRIPTEURS']
paths_take_all = ['GESTION',
'DONNEES/CLASSES',
'DONNEES/DESCRIPTEURS'
'DONNEES/PROCEDURES/LOTS'
'DONNEES/PROCEDURES/AUTRES'
]

In [4]:
def clean_pair(pair):
    new_pair = ['','']
    for i in range(len(pair)):
        new_pair[i] = pair[i].replace(',', '')
        new_pair[i] = new_pair[i].replace('\n', '')
    return new_pair

In [5]:
def xml_parser(paths_common, paths_take_all, file):
    pairs=[]
    for path in paths_take_all:
        element = file.find(path)
        if element is not None:
            pairs.append(clean_pair([element.tag, element.text]))
    for path in paths_common:
        if file.find(path) is not None: 
            for element in file.find(path):
                #print(element)
                if element.tag and element.text is not None:
                    pairs.append(clean_pair([element.tag, element.text]))
    return pairs

In [6]:
def write_to_csv(big_pair_list, file_name):
    bar3 = pbar(len(big_pair_list))
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=',', quotechar=' ',
                            quoting=csv.QUOTE_ALL)
        for pair_list in big_pair_list:
            bar3.update(big_pair_list.index(pair_list))
            
            #print(pair_list)
            tags = []
            text = []
            for pair in pair_list:
                tags.append(pair[0])
                text.append(pair[1])
            spamwriter.writerow(tags)
            spamwriter.writerow(text)

In [7]:
def pbar(maxlen):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [12]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]
    
    for i in range(len(years)):
        paths = glob.glob('{}/mp*/*.xml'.format(years[i]))
        annonces = []
    
        bar = pbar(len(paths))
    
        for path in paths:
            bar.update(paths.index(path))
            file = open(path, encoding='ISO-8859-1')
            apple = file.read()
            apple = apple.replace('&apos;', "'")
            apple = apple.replace('&deg;', '°')
            apple = apple.replace('&quot;', '"')
            tree = ET.fromstring(apple)
            a = tree.getchildren()
    
            annonces.append(a)
    
        p = []
        bar2 = pbar(len(annonces))
        for annonce in annonces:
            bar2.update(annonces.index(annonce))
            for j in range(len(annonce)):
                p.append(xml_parser(paths_common, paths_take_all, annonce[j]))
        write_to_csv(p, year)

    

In [ ]:
write_years2csv(2005, 2005)

[====================                                  ]  38% (ETA:   0:00:18) 